<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/GPT_3_ARPs_and_ProgrammingPosts_Classifiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://chatgpt.com/share/11cbff83-ed2c-416b-8c99-b962388a075b

In [18]:
#pip install openai --upgrade

In [ ]:
#!openai migrate

In [ ]:
!pip install datasets

In [1]:
import openai
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [6]:
# Load your dataset
data=pd.read_excel('ARPs_and_ProgrammingPosts.xlsx')

In [ ]:
# Preprocess the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_function(examples):
    return tokenizer(examples, truncation=True, padding='max_length', max_length=128)


In [7]:
# Apply preprocessing
tokenized_inputs = data['Question_body'].apply(preprocess_function)


In [8]:
# Convert tokenized inputs to a DataFrame
tokenized_df = pd.DataFrame(tokenized_inputs.tolist())

In [9]:
# Add the labels to the tokenized DataFrame
tokenized_df['labels'] = data['Label']

In [10]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(tokenized_df, test_size=0.2)


In [11]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [12]:
# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
# Metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)